## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import h5py
import deepdish as dd

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import logomaker

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [2]:
variant_table_dir = '/oak/stanford/groups/akundaje/projects/neuro-variants/variant_tables/asd/K562_bias'
variant_shap_dir = '/oak/stanford/groups/akundaje/projects/neuro-variants/variant_shap/asd/K562_bias'
variant_plot_dir = '/oak/stanford/groups/akundaje/projects/neuro-variants/variant_plots/asd/K562_bias'
variant_list_dir = '/oak/stanford/groups/akundaje/projects/neuro-variants/variant_lists'

shap_inputs_file = variant_list_dir + '/neuro.asd.variants.shap.tsv'

In [3]:
clusters = sorted([i for i in os.listdir(variant_shap_dir + '/corces_2020') + os.listdir(variant_shap_dir + '/trevino_2021') + os.listdir(variant_shap_dir + '/domcke_2020')])
folds = ['fold_' + str(i) for i in range(5)]

print(len(clusters))
print()

print([i for i in clusters])
print()

19

['corces_2020.Cluster1', 'corces_2020.Cluster11', 'corces_2020.Cluster13', 'corces_2020.Cluster19', 'corces_2020.Cluster24', 'corces_2020.Cluster8', 'domcke_2020.fetal_brain.Astrocytes', 'domcke_2020.fetal_brain.Excitatory_neurons', 'domcke_2020.fetal_brain.Inhibitory_neurons', 'domcke_2020.fetal_heart.Cardiomyocytes', 'domcke_2020.fetal_heart.Endocardial_cells', 'domcke_2020.fetal_heart.Myeloid_cells', 'domcke_2020.fetal_heart.Smooth_muscle_cells', 'domcke_2020.fetal_heart.Stromal_cells', 'domcke_2020.fetal_heart.Vascular_endothelial_cells', 'trevino_2021.c10', 'trevino_2021.c11', 'trevino_2021.c15', 'trevino_2021.c19']



In [4]:
shap_inputs = pd.read_table(shap_inputs_file, names=['chr_hg38', 'pos_hg38', 'ref_allele', 'alt_allele', 'variant_id'])

shap_inputs

,chr_hg38,pos_hg38,ref_allele,alt_allele,variant_id
0,chr7,148230432,G,T,chr7:148230432:G:T
1,chr14,27886425,C,T,chr14:27886425:C:T


In [5]:
def softmax(x, temp=1):
    norm_x = x - np.mean(x, axis=1, keepdims=True)
    return np.exp(temp*norm_x)/np.sum(np.exp(temp*norm_x), axis=1, keepdims=True)

In [6]:
allele1_preds = {}
allele2_preds = {}

for dataset in ['corces_2020', 'trevino_2021', 'domcke_2020']:
    print(dataset)

    for cluster in [i for i in clusters if i.startswith(dataset)]:
        print(cluster)
        allele1_pred_counts = {}
        allele2_pred_counts = {}

        allele1_pred_profile = {}
        allele2_pred_profile = {}
        
        allele1_preds[cluster] = {}
        allele2_preds[cluster] = {}

        for fold in folds:
            infile = h5py.File(variant_shap_dir + '/' + dataset + '/' + cluster + '/' + str(fold) + '/' + cluster + '.' + str(fold) + '.asd.shap.variant_predictions.h5')
            
            allele1_pred_counts[fold] = np.array(infile['observed']['allele1_pred_counts'])
            allele2_pred_counts[fold] = np.array(infile['observed']['allele2_pred_counts'])
            
            allele1_pred_profile[fold] = np.array(infile['observed']['allele1_pred_profiles'])
            allele2_pred_profile[fold] = np.array(infile['observed']['allele2_pred_profiles'])
            
            allele1_preds[cluster][fold] = allele1_pred_counts[fold] * softmax(allele1_pred_profile[fold])
            allele2_preds[cluster][fold] = allele2_pred_counts[fold] * softmax(allele2_pred_profile[fold])
            
        allele1_pred_counts['mean'] = np.mean(np.array([allele1_pred_counts[fold] for fold in folds]), axis=0)
        allele2_pred_counts['mean'] = np.mean(np.array([allele2_pred_counts[fold] for fold in folds]), axis=0)
        
        allele1_pred_profile['mean'] = np.mean(np.array([allele1_pred_profile[fold] for fold in folds]), axis=0)
        allele2_pred_profile['mean'] = np.mean(np.array([allele2_pred_profile[fold] for fold in folds]), axis=0)
        
        allele1_preds[cluster]['mean'] = np.mean(np.array([allele1_preds[cluster][fold] for fold in folds]), axis=0)
        allele2_preds[cluster]['mean'] = np.mean(np.array([allele2_preds[cluster][fold] for fold in folds]), axis=0)

corces_2020
corces_2020.Cluster1
corces_2020.Cluster11
corces_2020.Cluster13
corces_2020.Cluster19
corces_2020.Cluster24
corces_2020.Cluster8
trevino_2021
trevino_2021.c10
trevino_2021.c11
trevino_2021.c15
trevino_2021.c19
domcke_2020
domcke_2020.fetal_brain.Astrocytes
domcke_2020.fetal_brain.Excitatory_neurons
domcke_2020.fetal_brain.Inhibitory_neurons
domcke_2020.fetal_heart.Cardiomyocytes
domcke_2020.fetal_heart.Endocardial_cells
domcke_2020.fetal_heart.Myeloid_cells
domcke_2020.fetal_heart.Smooth_muscle_cells
domcke_2020.fetal_heart.Stromal_cells
domcke_2020.fetal_heart.Vascular_endothelial_cells


In [7]:
allele1_shap = {'counts': {}, 'profile': {}}
allele2_shap = {'counts': {}, 'profile': {}}

In [8]:
for dataset in ['corces_2020', 'trevino_2021', 'domcke_2020']:
    for score_type in ['counts']: #, 'profile']:
        for cluster in [i for i in clusters if i.startswith(dataset)]:
            print(cluster)
            
            if cluster not in allele2_shap[score_type]:
                allele1_shap[score_type][cluster] = {}
                allele2_shap[score_type][cluster] = {}
                
            if len(allele2_shap[score_type][cluster]) < 5:
                for fold in folds:
                    # print(fold)
                    infile = dd.io.load(variant_shap_dir + '/' + dataset + '/' + cluster + '/' + str(fold) + '/' + cluster + '.' + str(fold) + '.asd.shap.variant_shap.counts.h5')
                    alleles = np.array(infile['alleles'])
                    allele1_shap[score_type][cluster][fold] = np.array(infile['projected_shap']['seq'])[alleles==0]
                    allele2_shap[score_type][cluster][fold] = np.array(infile['projected_shap']['seq'])[alleles==1]

                allele1_shap[score_type][cluster]['mean'] = np.mean(np.array([allele1_shap[score_type][cluster][fold] for fold in folds]), axis=0)
                allele2_shap[score_type][cluster]['mean'] = np.mean(np.array([allele2_shap[score_type][cluster][fold] for fold in folds]), axis=0)

corces_2020.Cluster1
corces_2020.Cluster11
corces_2020.Cluster13
corces_2020.Cluster19
corces_2020.Cluster24
corces_2020.Cluster8
trevino_2021.c10
trevino_2021.c11
trevino_2021.c15
trevino_2021.c19
domcke_2020.fetal_brain.Astrocytes
domcke_2020.fetal_brain.Excitatory_neurons
domcke_2020.fetal_brain.Inhibitory_neurons
domcke_2020.fetal_heart.Cardiomyocytes
domcke_2020.fetal_heart.Endocardial_cells
domcke_2020.fetal_heart.Myeloid_cells
domcke_2020.fetal_heart.Smooth_muscle_cells
domcke_2020.fetal_heart.Stromal_cells
domcke_2020.fetal_heart.Vascular_endothelial_cells


In [9]:
from multiprocessing import Pool

def parallel_func(df, func, n_cores=40):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    
    return df

In [10]:
for dataset in ['corces_2020', 'trevino_2021', 'domcke_2020']:
    for cluster in [i for i in clusters if i.startswith(dataset)]:
        os.makedirs(variant_plot_dir + '/' + dataset + '/' + cluster, exist_ok=True)

In [11]:
def plot_variants(df, score_type='counts', fold='mean'):
    for index,row in df.iterrows():
        for dataset in ['corces_2020', 'trevino_2021', 'domcke_2020']:
            for cluster in [i for i in clusters if i.startswith(dataset)]:
                outfile = variant_plot_dir + '/' + dataset + '/' + cluster + '/' + row['variant_id'] + '.png'

                if not os.path.isfile(outfile) and not pd.isna(row['variant_id']):

                    flank = 150
                    x = np.arange(-flank, flank)
                    c = np.zeros(flank * 2)

                    fig = figure(figsize=(20, 9))
                    ax1 = fig.add_subplot(311)
                    plt.plot(x, c, color='black')

                    plt.axvline(x=0, color='black', ls='--', linewidth=1)
                    ax1.margins(x=0, y=0.1)  # Remove padding
                    ax1.set_xlim(-flank, flank + 1)
                    plt.xticks([i for i in range(-flank, flank+1, 50)])

                    plt.title(row['variant_id'] + ' (' + row['ref_allele'] + '/' + row['alt_allele'] + ') ' + '--- ' + cluster,
                                fontsize=18, weight='bold')

                    ylim = [np.min(np.array([np.min(allele1_shap[score_type][cluster][fold][index][:,1057-flank:1057+flank].T),
                                                np.min(allele2_shap[score_type][cluster][fold][index][:,1057-flank:1057+flank].T)])) * 1.1,
                            np.max(np.array([np.max(allele1_shap[score_type][cluster][fold][index][:,1057-flank:1057+flank].T),
                                                np.max(allele2_shap[score_type][cluster][fold][index][:,1057-flank:1057+flank].T)])) * 1.1]
                    
                    allele1_label = 'ref allele (' + row['ref_allele'] + ')'
                    allele2_label = 'alt allele (' + row['alt_allele'] + ')'

                    ax1.tick_params(axis='x', labelsize=16)
                    ax1.tick_params(axis='y', labelsize=16)

                    plt.plot(x, allele1_preds[cluster][fold][index][500-flank:500+flank], color='royalblue', label=allele1_label) #, linewidth=1)
                    plt.plot(x, allele2_preds[cluster][fold][index][500-flank:500+flank], color='firebrick', label=allele2_label) #, linewidth=1)
                    plt.legend(prop={'size': 18}, loc='upper right')

                    flank = 150

                    ax2 = fig.add_subplot(312)
                    logo1 = logomaker.Logo(pd.DataFrame(allele1_shap[score_type][cluster][fold][index][:,1057-flank:1057+flank].T,
                                                        columns=['A','C','G','T']), ax=ax2)
                    logo1.ax.set_xlim(0, (flank * 2) + 1)
                    logo1.ax.set_ylim(ylim)
                    ax2.axvline(x=flank, color='black', ls='--', linewidth=1)
                    logo1.ax.set_xticks([i for i in range(0, (flank * 2) + 1, 50)])
                    logo1.ax.set_xticklabels([str(i) for i in range(-flank, flank+1, 50)])

                    plt.text(0.987, 0.90, allele1_label, 
                                verticalalignment='top', horizontalalignment='right',
                                transform=ax2.transAxes, size=18,
                                bbox=dict(boxstyle='square,pad=0.5', facecolor='white', edgecolor='black'))

                    ax2.tick_params(axis='x', labelsize=16)
                    ax2.tick_params(axis='y', labelsize=16)

                    ax3 = fig.add_subplot(313)
                    logo2 = logomaker.Logo(pd.DataFrame(allele2_shap[score_type][cluster][fold][index][:,1057-flank:1057+flank].T,
                                                        columns=['A','C','G','T']), ax=ax3)
                    logo2.ax.set_xlim(0, (flank * 2) + 1)
                    logo2.ax.set_ylim(ylim)
                    ax3.axvline(x=flank, color='black', ls='--', linewidth=1)
                    logo2.ax.set_xticks([i for i in range(0, (flank * 2) + 1, 50)])
                    logo2.ax.set_xticklabels([str(i) for i in range(-flank, flank+1, 50)])

                    plt.text(0.987, 0.90, allele2_label, 
                                verticalalignment='top', horizontalalignment='right',
                                transform=ax3.transAxes, size=18,
                                bbox=dict(boxstyle='square,pad=0.5', facecolor='white', edgecolor='black'))
                    
                    ax3.tick_params(axis='x', labelsize=16)
                    ax3.tick_params(axis='y', labelsize=16)

                    plt.subplots_adjust(wspace=0.3, hspace=0.4)
                    fig.tight_layout()
                    plt.savefig(outfile)
                    plt.close()
    return df

In [12]:
parallel_func(shap_inputs, plot_variants)

,chr_hg38,pos_hg38,ref_allele,alt_allele,variant_id
0,chr7,148230432,G,T,chr7:148230432:G:T
1,chr14,27886425,C,T,chr14:27886425:C:T
